In [92]:
#downloading and appending stock price
# interval: 1m, 2m, 1h, 1d etc.
#tickers=["ac.to","aapl", "goog", "amzn", "BAC", "BA"]

def get_stock_yf(tickers, interval_time, startdate, enddate):
    import yfinance as yf
    import pandas as pd
    from datetime import date, timedelta

    pd.options.mode.chained_assignment = None

    stock=pd.DataFrame()
    # print(ticker_funds)

    # download us stock price
    stock_df = yf.download(tickers, group_by=tickers, interval=interval_time, start=startdate, end=None)
    stock_df.index.name='Datetime'

    #us_stock_price.index = pd.to_datetime(us_stock_price.index, format='%Y-%m-%d')


    for ticker in tickers:
        stock_data = stock_df.loc[:,ticker.upper()]
        stock_data.loc[:,'ticker'] = ticker.upper()
        stock = pd.concat([stock, stock_data])

    stock.reset_index(inplace=True)
    stock['Datetime']=stock['Datetime'].dt.tz_localize(None)
    stock=stock[(stock['Volume'].notnull()) & (stock['Volume']!=0)]
    stock['date']=stock['Datetime'].dt.date

    # Calculate intra-day return in bps
    stock['IntraDay Return'] = (stock['Close'] - stock['Open']) / stock['Open'] * 100
    
    return stock

In [93]:
# # get_stock_yf(tickers, interval_time, startdate) download stock data for a series of tickers, and adding a intraday return
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

path = '/Users/jayren/Desktop/stock/Stock_Daily'
dailydata = '/Users/jayren/Desktop/stock/stock_Daily/dailydata'
consolidated = '/stockPrice_consolidated'


# list of tickers
us_sp500_list = pd.read_excel(path + '/list.xlsx', sheet_name='SP500_LIST')
ca_sptsx_list = pd.read_excel(path + '/list.xlsx', sheet_name='SPTSX_LIST')
adhoc_list = pd.read_excel(path + '/list.xlsx', sheet_name='Adhoc')

us_t = us_sp500_list['Symbol'].tolist()
ca_t = ca_sptsx_list['Symbol'].tolist()
adhoc_t = adhoc_list['Symbol'].tolist()

tickers=['googl', 'nvda', 'msft', 'amzn', 'aapl', 'meta', 'tsla', 'v', 'jpm', 'unh', 'hd', 'pg', 'dis', 'nflx', 'intc', 'csco', 'cmcsa']
tickers = [ticker.upper() for ticker in tickers]
#tickers=us_t

interval='1d'
startdate='2020-01-01'
enddate='2025-12-25'

df=get_stock_yf(tickers, interval,startdate, enddate)

window = 10  # rolling window size
tickers = ['GOOGL', 'TSLA']
tickers = [t.upper() for t in tickers]

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
np.set_printoptions(threshold=np.inf)

filtered_df = df[df['ticker'].isin(tickers)] 
# Group by date and pivot to get intra-day returns for TD and BMO in separate columns 
data = filtered_df.pivot_table(index='date', columns='ticker', values='Close')

# Storage
s1_series = []
adf_stats = []
p_values = []
autocorr_dict = {lag: [] for lag in range(1, 6)}
dates = data.index[window:]

# Fit a VAR model, Vector AutoRegression model of order 1
# Rolling VAR loop
for i in range(window, len(data)):
    try:
        window_data = data.iloc[i - window:i]
        model = VAR(window_data)
        results = model.fit(1)

        # Extract coefficient matrix B and compute K = I - B
        B = results.params.values[1:, :]
        I = np.eye(B.shape[0])
        K = I - B

        # Eigen-decomposition
        eigenvalues, eigenvectors = np.linalg.eig(K)
        sorted_indices = np.argsort(np.abs(eigenvalues))[::-1]
        eigenvectors = eigenvectors[:, sorted_indices]
        v1 = eigenvectors[:, 0]  # dominant eigenvector

        # Compute co-integration factor S1
        current_prices = data.iloc[i].values
        s1 = np.dot(current_prices, v1)
        # If s1_value is complex, take the real part only
        #s1_value = np.real(s1_value)
        s1_series.append(s1)

        # ADF test on rolling spread (optional — just last N S1s)
        recent_spread = np.dot(window_data.values, v1)
        adf_result = adfuller(recent_spread)
        adf_stats.append(adf_result[0])
        p_values.append(adf_result[1])

        # Autocorrelation
        for lag in range(1, 6):
            if len(recent_spread) > lag:
                r = np.corrcoef(recent_spread[:-lag], recent_spread[lag:])[0, 1]
            else:
                r = np.nan
            autocorr_dict[lag].append(r)

    except Exception as e:
        s1_series.append(np.nan)
        adf_stats.append(np.nan)
        p_values.append(np.nan)
        for lag in autocorr_dict:
            autocorr_dict[lag].append(np.nan)

# --- Create Final Output ---
result_df = pd.DataFrame({
    'date': dates,
    'S1': s1_series,
    'ADF Statistic': adf_stats,
    'p-value': p_values,
})
for lag in autocorr_dict:
    result_df[f'autocorr_lag{lag}'] = autocorr_dict[lag]

# Normalize for z-score if desired
result_df['S1_z'] = (result_df['S1'] - result_df['S1'].mean()) / result_df['S1'].std()

# Final result
print(result_df.tail())

result_df=result_df.apply(np.real)


# Step 4: Diagonalize K
# Compute eigenvalues and eigenvectors of K
#eigenvalues, eigenvectors = np.linalg.eig(K)

# Sort eigenvalues and corresponding eigenvectors by the absolute value of eigenvalues
#sorted_indices = np.argsort(np.abs(eigenvalues))[::-1]
# [::-1] This reverses the order — so now you get indices from largest to smallest magnitude.
#So np.argsort(np.abs(eigenvalues)) gives indices for smallest to largest absolute eigenvalue.

#eigenvalues = eigenvalues[sorted_indices]
#Reorders the eigenvalues so the largest (by magnitude) comes first.
#eigenvectors = eigenvectors[:, sorted_indices]
#	•	Each eigenvalue corresponds to one eigenvector.
#	•	By sorting from strongest to weakest (in terms of magnitude), you’re putting the most impactful component first.

# Step 5: Construct co-integration factor S1
# Transform the original prices using the first eigenvector (strongest mean-reverting component)
# prices = data.values  # Original price data matrix
#S1 = np.dot(prices, eigenvectors[:, 0])  # Co-integration factor (first eigenvector)

# print('eigenvectors for s1 is: ', eigenvectors[:,0])

#Check Stationarity of S1

print("If p-value < 0.05, you can reject the null hypothesis of a unit root → series is stationary.")

# # Step 6: Analyze the co-integration factor
# # Compute autocorrelation of S1 to confirm mean-reverting behavior
# def autocorrelation(series, lag):
#     return np.corrcoef(series[:-lag], series[lag:])[0, 1]

# lags = range(1, 6)  # Test lags from 1 to 5
# autocorrelations = [autocorrelation(S1, lag) for lag in lags]

# # Display results
# print("Eigenvalues:", eigenvalues)
# print("Autocorrelations of S1:", autocorrelations)



# data.reset_index(inplace=True)


[*********************100%***********************]  17 of 17 completed
/Users/jayren/Desktop/Pair Trading/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency B will be used.

/Users/jayren/Desktop/Pair Trading/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency B will be used.

/Users/jayren/Desktop/Pair Trading/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency B will be used.

/Users/jayren/Desktop/Pair Trading/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/jayren/Desktop/Pair Trading/.venv/lib/python3.13/site-packages/sta

            date                      S1  ADF Statistic   p-value  \
1349  2025-05-30   78.757378+  0.000000j      -1.229765  0.660662   
1350  2025-06-02   61.768496+  0.000000j      -2.512695  0.112416   
1351  2025-06-03   62.766975+  0.000000j      -3.588450  0.005986   
1352  2025-06-04   92.847259+  0.000000j      -3.871684  0.002253   
1353  2025-06-05  103.870192+  0.000000j      -3.192444  0.020426   

           autocorr_lag1       autocorr_lag2       autocorr_lag3  \
1349  0.416757+0.000000j  0.162425+0.000000j  0.021875+0.000000j   
1350  0.171042+0.000000j  0.038081+0.000000j -0.763287+0.000000j   
1351  0.093392+0.000000j -0.592675+0.000000j -0.669739+0.000000j   
1352 -0.177809+0.000000j -0.652277+0.000000j -0.054259+0.000000j   
1353 -0.134208+0.000000j -0.505362+0.000000j  0.337008+0.000000j   

           autocorr_lag4       autocorr_lag5                S1_z  
1349 -0.507402+0.000000j -0.645364+0.000000j  0.372240-0.004247j  
1350 -0.233865+0.000000j  0.108426+0.00000

In [94]:
# plot the z-score
#Z-score in pairs trading is a statistical measure that tells you how far the current value of the spread 
# is from its historical mean, measured in standard deviations.
#Z-score Interpretation:
#Action
#Z > +2 Spread is too high, Short the spread (e.g., sell EDU, buy AMC)
# Z < -2 Spread is too low Long the spread (e.g., buy EDU, sell AMC)
# Z near 0 Spread is normal Close positions or stay out
# This strategy assumes that the spread is mean-reverting, so extreme deviations are opportunities to bet on a return to the average

# exit your current position when the z-score is between -0.5 and 0.5

# Calculate rolling mean and std for S1

import pandas as pd
import plotly.graph_objects as go

# Step 1: Use z-score from result_df (assumes it has 'date' and 'S1_z')
result_df = result_df.dropna(subset=['S1_z'])  # ensure no NaNs
z_score = result_df.set_index('date')['S1_z']  # use date as index

# Step 2: Define thresholds and initialize signals
entry_threshold = 2.0
exit_threshold = 0.5
signals = pd.Series(0, index=z_score.index)

# Step 3: Apply signal rules
signals.loc[z_score > entry_threshold] = -1  # Sell
signals.loc[z_score < -entry_threshold] = 1  # Buy

# Optional: Add to result_df if needed for backtest later
result_df['signal'] = signals.values

# Step 4: Extract buy/sell signal points
buy_signals = z_score[signals == 1]
sell_signals = z_score[signals == -1]

# Step 5: Plot with Plotly
fig = go.Figure()

# Z-score time series
fig.add_trace(go.Scatter(
    x=z_score.index, y=z_score,
    mode='lines', name='Z-Score',
    line=dict(color='purple'),
    hovertemplate='%{x|%Y-%m-%d}<br>Z-Score: %{y:.2f}<extra></extra>'
))

# Entry/exit thresholds
fig.add_trace(go.Scatter(
    x=z_score.index, y=[entry_threshold]*len(z_score),
    mode='lines', name='Entry Threshold',
    line=dict(color='green', dash='dash')
))
fig.add_trace(go.Scatter(
    x=z_score.index, y=[-entry_threshold]*len(z_score),
    mode='lines', showlegend=False,
    line=dict(color='green', dash='dash')
))
fig.add_trace(go.Scatter(
    x=z_score.index, y=[exit_threshold]*len(z_score),
    mode='lines', name='Exit Threshold',
    line=dict(color='orange', dash='dash')
))
fig.add_trace(go.Scatter(
    x=z_score.index, y=[-exit_threshold]*len(z_score),
    mode='lines', showlegend=False,
    line=dict(color='orange', dash='dash')
))

# Buy/Sell signal markers
fig.add_trace(go.Scatter(
    x=buy_signals.index, y=buy_signals,
    mode='markers', name='Buy Signal',
    marker=dict(symbol='triangle-up', color='green', size=12),
    hovertemplate='%{x|%Y-%m-%d}<br>Buy: %{y:.2f}<extra></extra>'
))
fig.add_trace(go.Scatter(
    x=sell_signals.index, y=sell_signals,
    mode='markers', name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=12),
    hovertemplate='%{x|%Y-%m-%d}<br>Sell: %{y:.2f}<extra></extra>'
))

# Final layout settings
fig.update_layout(
    title="Z-Score of Co-integration Factor (Rolling VAR)",
    xaxis_title="Date",
    yaxis_title="Z-Score",
    template="plotly_white",
    height=500,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show(renderer="browser")

In [95]:
#testing new code including s1 line, and so on

# Recalculate rolling mean and std on S1

import numpy as np
import plotly.graph_objects as go
import pandas as pd

# Ensure 'date' is datetime and S1 is clean
result_df['date'] = pd.to_datetime(result_df['date'])
S1_series = result_df.set_index('date')['S1']

# Compute rolling mean and std on S1
rolling_window = 20
rolling_mean = S1_series.rolling(window=rolling_window).mean()
rolling_std = S1_series.rolling(window=rolling_window).std()

upper_band = rolling_mean + 2 * rolling_std
lower_band = rolling_mean - 2 * rolling_std

# Generate signals
signals = pd.Series(0, index=S1_series.index)
signals[S1_series > upper_band] = -1  # Sell
signals[S1_series < lower_band] = 1   # Buy

buy_signals = S1_series[signals == 1]
sell_signals = S1_series[signals == -1]

# Initialize plot
fig = go.Figure()

# S1 line
fig.add_trace(go.Scatter(
    x=S1_series.index, y=S1_series,
    mode='lines', name='S1 (Spread)',
    line=dict(color='black'),
    hovertemplate='%{x|%Y-%m-%d}<br>S1: %{y:.2f}<extra></extra>'
))

# Mean line
mean_s1 = S1_series.mean()
fig.add_trace(go.Scatter(
    x=S1_series.index, y=[mean_s1]*len(S1_series),
    mode='lines', name='Mean of S1',
    line=dict(color='red', dash='dash')
))

# Rolling mean
fig.add_trace(go.Scatter(
    x=S1_series.index, y=rolling_mean,
    mode='lines', name='Rolling Mean (20d)',
    line=dict(color='blue', dash='dash')
))

# ±2σ band shaded region
fig.add_trace(go.Scatter(
    x=pd.concat([pd.Series(S1_series.index), pd.Series(S1_series.index[::-1])]),
    y=pd.concat([upper_band, lower_band[::-1]]),
    fill='toself',
    fillcolor='rgba(173,216,230,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name='±2σ Band'
))

# Buy/Sell markers
fig.add_trace(go.Scatter(
    x=buy_signals.index, y=buy_signals,
    mode='markers', name='Buy Signal',
    marker=dict(symbol='triangle-up', color='green', size=12),
    hovertemplate='%{x|%Y-%m-%d}<br>Buy: %{y:.2f}<extra></extra>'
))
fig.add_trace(go.Scatter(
    x=sell_signals.index, y=sell_signals,
    mode='markers', name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=12),
    hovertemplate='%{x|%Y-%m-%d}<br>Sell: %{y:.2f}<extra></extra>'
))

# Layout
fig.update_layout(
    title=f"Rolling S1 Spread with ±2σ Band and Trading Signals ({tickers[0]} / {tickers[1]})",
    xaxis_title="Date",
    yaxis_title="S1 Value",
    template="plotly_white",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    hovermode="x unified"
)


#fig.write_html("S1_strategy_chart.html")  # Inline if running in a notebook
#Show the plot
fig.show(renderer="browser")

In [96]:
# The Johansen Test is a multivariate statistical test that identifies the co-integrating relationship 
# between two or more time series. It provides the eigenvectors that define the linear relationship.

# Steps:
# 1. Perform the Johansen co-integration test on the log prices of INTC and SMH.
# 2. Retrieve the eigenvector corresponding to the largest eigenvalue (indicating the co-integrating relationship).
# 3. Normalize the eigenvector to set one of the weights (usually 𝛽) to 1. The other weight (α) is scaled accordingly.

# Outcome:
# α: Weight for INTC.
# β: Weight for SMH.

# get_stock_yf(tickers, interval_time, startdate) download stock data for a series of tickers, and adding a intraday return

# tickers=['TD','bmo','nvda', 'aapl', 'rblx', 'intc', 'smh','IYG', 'RY']
# interval='1d'
# startdate='2010-01-01'

# df=get_stock_yf(tickers, interval,startdate)

# tickers=['BMO', 'RY']

filtered_df = df[df['ticker'].isin(tickers)] 
# Group by date and pivot to get intra-day returns for TD and BMO in separate columns 
data = filtered_df.pivot_table(index='date', columns='ticker', values='Close')


from statsmodels.tsa.vector_ar.vecm import coint_johansen
import numpy as np

# Assume we have log prices for INTC and SMH
log_price_ticker0 = np.log(data[tickers[0]])
log_price_ticker1 = np.log(data[tickers[1]])

# Combine the two series into a DataFrame
johnson_data = np.column_stack((log_price_ticker0, log_price_ticker1))

# Perform Johansen test
johansen_test = coint_johansen(johnson_data, det_order=0, k_ar_diff=1)

# Extract the eigenvector corresponding to the largest eigenvalue
eigenvector = johansen_test.evec[:, 0]  # First column of eigenvectors

# Normalize the eigenvector (set beta to 1)
beta = eigenvector[1]  # Weight for SMH
alpha = eigenvector[0] / beta  # Normalize INTC's weight relative to SMH

print(f"Alpha (ticker0 weight): {alpha}")
print(f"Beta (ticker1 weight): {beta}")

print(tickers)



# if both positive: meaning they move together in the same direction.
# if both negative: s1=-abs(alpha) *ticker0 -abs(beta) * ticker1 ,but the negative signs might indicate 
# that s1 is constructed as a deviation from some baseline.
# if opposite sign, that means s1 is constructed as the spread betwen two tickers, which is common in paires trading

# Check in Practice:
# To verify the signs in your analysis, you can:

# Look at the eigenvector or regression coefficients after deriving α and β.
# Plot the co-integration factor S1 to ensure it exhibits the desired mean-reverting behavior.
# Ensure that the signs make economic sense based on the relationship between the two assets.

Alpha (ticker0 weight): -0.848692149591854
Beta (ticker1 weight): -2.56387576089646
['GOOGL', 'TSLA']


In [97]:
# stock price over time

# get_stock_yf(tickers, interval_time, startdate) download stock data for a series of tickers, and adding a intraday return

#  Plot the stock price by date

# tickers=['TD','bmo', 'rblx', 'intc', 'SMH','IYG', 'RY', 'NVDA', 'AAPL','GOOG', 'AMZN',  'META', 'MSFT', 'TSLA']
# interval='1d'
# startdate='2010-01-01'

# df=get_stock_yf(tickers, interval,startdate)

# tickers=['AAPL', 'MSFT']

filtered_df = df[df['ticker'].isin(tickers)] 
# Group by date and pivot to get intra-day returns for TD and BMO in separate columns 
data = filtered_df.pivot_table(index='date', columns='ticker', values='Close')


# Create Plotly figure
import plotly.graph_objects as go
import plotly.express as px

data.columns.name=None
data=data.reset_index()
#data = data.rename(columns={'index': 'original_index'}) 

# Add annotations for the first and last data points
# first_date = df['Date'].iloc[0]
# first_value = df['Value'].iloc[0]

last_date = data['date'].iloc[-1]
ticker0_last_value = round(data[tickers[0]].iloc[-1],2)
ticker1_last_value = round(data[tickers[1]].iloc[-1],2)

color_mapping=[ (0, '#0000FF'), (1, 'red')]
# Create a new figure
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['date'], y=data[tickers[0]],
        mode='lines',
        name=tickers[0],
        line=dict(color='blue')))

fig.add_trace(go.Scatter(x=data['date'], y=data[tickers[1]],
        mode='lines',
        name=tickers[1],
        line=dict(color='red')))

fig.add_annotation(
    x=last_date, y=ticker0_last_value, 
    text=tickers[0] + f": ({last_date.strftime('%Y-%m-%d')}, {ticker0_last_value})", 
    showarrow=True, arrowhead=2)

fig.add_annotation(
    x=last_date, y=ticker1_last_value, 
    text=tickers[1] + f": ({last_date.strftime('%Y-%m-%d')}, {ticker1_last_value})", 
    showarrow=True, arrowhead=2)

fig.update_layout(xaxis=dict(tickformat="%Y-%m-%d"),
    title='stock price over time',
    xaxis_title='Date',
    yaxis_title='Price'
    #bargap=0
    )

#Show the plot
fig.show(renderer="browser")

In [98]:
# backtest Steps

# 1. Start with $10,000 cash and zero position.
# 2. When first Buy Signal (S1 < lower band), enter full position.
# 3. Hold position (do nothing) until reaching Exit Signal (S1 crosses back to mean Â±0.5Ïƒ).
# 4. Sell entire position when Exit Signal happens.
# 5. Now wait for next Buy Signal, repeat full portfolio deploy.
# 6. No Shorting for now unless explicitly stated.
# 7. Compute equity curve and Sharpe Ratio from cash movements.


import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp

# Assume you have S1, which is spread curve

# Recalculate rolling mean and std on S1
result_df['date'] = pd.to_datetime(result_df['date'])
S1_series = result_df.set_index('date')['S1']

rolling_mean = S1_series.rolling(window=20).mean()
rolling_std = S1_series.rolling(window=20).std()

upper_band = rolling_mean + rolling_std*2
lower_band = rolling_mean - rolling_std*2

# Generate buy/sell signals
signals = pd.Series(0, index=S1_series.index)
signals[S1_series > upper_band] = -1  # Sell signal
signals[S1_series < lower_band] = 1   # Buy signal

buy_signals = S1_series[signals == 1]
sell_signals = S1_series[signals == -1]


# Define Exit Signals
exit_band_upper = rolling_mean + rolling_std * 0.5
exit_band_lower = rolling_mean - rolling_std * 0.5
exit_signals = (S1_series > exit_band_lower) & (S1_series < exit_band_upper)

# Initialize
initial_cash = 10000
cash = initial_cash
position = 0  # 0 = no position, 1 = long
equity_curve = []
trade_log = []

current_value = cash
entry_price = None
entry_time = None

for i in range(1, len(S1_series)):
    current_time = S1_series.index[i]
    current_spread = S1_series.iloc[i]

    if position == 0:
        if signals.iloc[i] == 1:
            position = current_value / current_spread
            entry_price = current_spread
            entry_time = current_time
            cash = 0
    else:
        if exit_signals.iloc[i]:
        # if signals.iloc[i] == -1:
            cash = position * current_spread
            pnl = (current_spread - entry_price) * position
            trade_log.append({
                'Entry Date': entry_time,
                'Exit Date': current_time,
                'Entry Price': entry_price,
                'Exit Price': current_spread,
                'Profit': pnl
            })
            position = 0
            entry_price = None
            entry_time = None
    
    if position != 0:
        current_value = position * current_spread
    else:
        current_value = cash

    equity_curve.append(current_value)

# Equity series
equity_series = pd.Series(equity_curve, index=S1_series.index[1:])

# Returns
strategy_returns = equity_series.pct_change().fillna(0)

# Sharpe Ratio
mean_return = strategy_returns.mean()
std_return = strategy_returns.std()
sharpe_ratio = (mean_return / std_return) * np.sqrt(252)

# Maximum Drawdown
rolling_max = equity_series.cummax()
drawdown = (equity_series - rolling_max) / rolling_max
max_drawdown = drawdown.min()

# Trade log
trade_log_df = pd.DataFrame(trade_log)

# Find Buy/Sell points for plot
buy_dates = trade_log_df['Entry Date']
sell_dates = trade_log_df['Exit Date']
buy_prices = equity_series.loc[buy_dates]
sell_prices = equity_series.loc[sell_dates]

# Find major drawdown points (only < -20%)
major_drawdowns = drawdown[drawdown <= -0.2]

# Plot
fig = sp.make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    row_heights=[0.7, 0.3],
    subplot_titles=("Portfolio Value Over Time", "Drawdown Over Time")
)

# Portfolio Value line
fig.add_trace(go.Scatter(
    x=equity_series.index,
    y=equity_series,
    mode='lines',
    name='Portfolio Value',
    line=dict(color='green'),
    hovertemplate='%{x|%Y-%m-%d}<br>Portfolio Value: $%{y:.2f}<extra></extra>'
), row=1, col=1)

# Buy markers
fig.add_trace(go.Scatter(
    x=buy_dates,
    y=buy_prices,
    mode='markers',
    name='Buy',
    marker=dict(color='blue', size=10, symbol='triangle-up'),
    hovertemplate='%{x|%Y-%m-%d}<br>Buy: $%{y:.2f}<extra></extra>'
), row=1, col=1)

# Sell markers
fig.add_trace(go.Scatter(
    x=sell_dates,
    y=sell_prices,
    mode='markers',
    name='Sell',
    marker=dict(color='red', size=10, symbol='triangle-down'),
    hovertemplate='%{x|%Y-%m-%d}<br>Sell: $%{y:.2f}<extra></extra>'
), row=1, col=1)

# Initial Cash Line
fig.add_trace(go.Scatter(
    x=equity_series.index,
    y=[initial_cash] * len(equity_series),
    mode='lines',
    name='Initial Cash ($10,000)',
    line=dict(color='gray', dash='dash'),
    hoverinfo='skip'
), row=1, col=1)

# Drawdown Line
fig.add_trace(go.Scatter(
    x=drawdown.index,
    y=drawdown,
    mode='lines',
    name='Drawdown',
    line=dict(color='red'),
    hovertemplate='%{x|%Y-%m-%d}<br>Drawdown: %{y:.2%}<extra></extra>'
), row=2, col=1)

# Shade drawdown
fig.add_trace(go.Scatter(
    x=list(drawdown.index) + list(drawdown.index[::-1]),
    y=list(drawdown) + [0]*len(drawdown),
    fill='toself',
    fillcolor='rgba(255,0,0,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    showlegend=False
), row=2, col=1)

# Annotate major drawdowns (< -20%)

# for idx, dd in major_drawdowns.items():
#     fig.add_annotation(
#         x=idx,
#         y=equity_series.loc[idx],
#         text=f"{dd:.0%} Drawdown",
#         showarrow=True,
#         arrowhead=2,
#         arrowsize=1,
#         arrowwidth=2,
#         ax=0,
#         ay=-40,
#         font=dict(color="red"),
#         row=1, col=1
#     )

# Layout
fig.update_layout(
    height=900,
    width=1100,
    title="Backtest Results with Buy/Sell Markers and Major Drawdowns",
    hovermode="x unified",
    showlegend=True
)

fig.update_yaxes(title_text="Portfolio Value ($)", row=1, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show(renderer="browser")

print (sharpe_ratio, '\n')
print(trade_log_df)

-1.0533792005763423 

   Entry Date  Exit Date  Entry Price  Exit Price        Profit
0  2020-02-14 2020-02-19    17.850040   57.727043  2.234001e+04
1  2020-02-25 2020-02-26   -59.233837   44.847699 -5.682559e+04
2  2020-04-13 2020-04-15    20.637547   36.200513 -1.846480e+04
3  2020-05-07 2020-05-12   -85.513567   32.954379  5.950218e+04
4  2020-07-22 2020-07-27   -95.012874   32.672871 -2.224360e+04
5  2020-08-05 2020-08-12   -65.167301   45.647216  9.678698e+03
6  2020-09-21 2020-09-24   -83.095251   43.308967 -6.064848e+03
7  2020-10-12 2020-10-15  -128.665309    2.472488  2.117886e+03
8  2020-12-15 2020-12-30  -135.524529  -95.252978 -1.186559e+01
9  2021-01-22 2021-01-27  -245.794543  -15.434735 -2.630293e+01
10 2021-03-23 2021-03-30    65.276222   88.213622  6.192781e-01
11 2021-04-05 2021-04-14  -114.054720   -0.619122 -2.368718e+00
12 2021-04-26 2021-05-11  -177.223210   74.501408 -1.836306e-02
13 2021-06-11 2021-06-15  -230.589563   36.744621  6.300842e-03
14 2021-07-07 2021